In [1]:
import os
os.chdir('../../src/models')

In [2]:
from tqdm import tqdm
import simulate_pandemic as simp
from joblib import Parallel, delayed 
import numpy as np
import pandas as pd

Loading Graph... Done!


In [7]:
def sim_pandemic():
    restrictions = {'work': 0,  'school': 0,  'home': 0, 'neighbor': 0}
    pop_matrix = simp.init_infection(.0001)
    data = []
    for day in range(1, 200):
        if day >= 28:          
            restrictions = {'work': .6,  'school': 1,  'home': 0, 'neighbor': .6}
        if day >= 90:
            restrictions = {'work': 0,  'school': 0,  'home': 0, 'neighbor': 0}
            
        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:, 1])
        
    return data

In [8]:
data_sims = Parallel(n_jobs=-1)(delayed(sim_pandemic)()for i in tqdm(range(18)))

100%|██████████| 18/18 [00:51<00:00,  2.87s/it]


In [12]:
import plotly.graph_objects as go
import datetime

def plot_average_curve(sims, title, absolute=False, capacity=False):
    fig = go.Figure()
      
        
    values = []
    for data in data_sims:
        values.append([np.sum(d == 2) for d in data])
    values = np.array(values)
    values  =  values/55482 
    if absolute:
        values = values*12e6
    
    y = values.mean(axis=0)
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(y))    
    sd = values.std(axis=0)

        
    print(f'Mean at day 70 {y[-1]}')
    print(f'Sd at day 70 {sd[-1]}')
    
    fig.add_trace(go.Scatter(x=x, y=y,
        fill=None,
        mode='lines',
        line_color='indigo',
         name="Average Curve"
        ))

    fig.add_trace(go.Scatter(x=x, y=y+sd,
        fill=None,
        mode='lines',
        line_color='magenta',
        showlegend=False
       ))

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',      name="Standard Deviation"))

    fig.add_trace(go.Scatter(x=x, y=y-sd,
        fill=None,
        mode='lines',
        line_color='magenta',    showlegend=False
        ))


    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',     showlegend=False
    ))
    
    if capacity:
        fig.add_trace(go.Scatter(
            x=x,
            y=len(x)*[0.0025],
            mode='lines', line_color='yellow', name='Capacity',    showlegend=True
        ))

    fig.update_layout(title = title)

    fig.show()

In [15]:
plot_average_curve(data_sims, 'Infected', absolute=False)

Mean at day 70 0.014699462087804253
Sd at day 70 0.0015812706031242303
